<a href="https://colab.research.google.com/github/greyhound101/IEEE-CIS-Fraud/blob/master/finding_magic_feature_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
os.environ['KAGGLE_USERNAME'] = "tapaskd123" # username from the json file
os.environ['KAGGLE_KEY'] = "aba8dc1f085221111d925003fe5a88ed" # key from the json file
!kaggle competitions download -c ieee-fraud-detection

  0% 0.00/3.26M [00:00<?, ?B/s]
100% 3.26M/3.26M [00:00<00:00, 53.9MB/s]
  0% 0.00/1.14M [00:00<?, ?B/s]
100% 1.14M/1.14M [00:00<00:00, 157MB/s]
 84% 49.0M/58.3M [00:00<00:00, 58.2MB/s]
100% 58.3M/58.3M [00:00<00:00, 85.5MB/s]
 94% 49.0M/52.2M [00:00<00:00, 51.9MB/s]
100% 52.2M/52.2M [00:00<00:00, 59.7MB/s]
  0% 0.00/3.21M [00:00<?, ?B/s]
100% 3.21M/3.21M [00:00<00:00, 106MB/s]


In [2]:
from google.colab import drive
drive.mount('/content/gdrive')
drive.mount("/content/gdrive", force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive
Mounted at /content/gdrive


In [3]:
import pandas as pd
import numpy as np
trn=pd.read_csv('train_transaction.csv.zip')
trn_idn=pd.read_csv('train_identity.csv.zip')

In [4]:
trn=trn.merge(trn_idn,on='TransactionID',how='left')
import gc
del([trn_idn])
gc.collect()

11

In [5]:
trn.head()

,TransactionID,isFraud,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,card6,addr1,addr2,dist1,dist2,P_emaildomain,R_emaildomain,C1,C2,C3,C4,C5,C6,C7,C8,C9,C10,C11,C12,C13,C14,D1,D2,D3,D4,D5,D6,D7,D8,D9,...,id_01,id_02,id_03,id_04,id_05,id_06,id_07,id_08,id_09,id_10,id_11,id_12,id_13,id_14,id_15,id_16,id_17,id_18,id_19,id_20,id_21,id_22,id_23,id_24,id_25,id_26,id_27,id_28,id_29,id_30,id_31,id_32,id_33,id_34,id_35,id_36,id_37,id_38,DeviceType,DeviceInfo
0,2987000,0,86400,68.5,W,13926,NaN,150.0,discover,142.0,credit,315.0,87.0,19.0,NaN,NaN,NaN,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,2.0,0.0,1.0,1.0,14.0,NaN,13.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2987001,0,86401,29.0,W,2755,404.0,150.0,mastercard,102.0,credit,325.0,87.0,NaN,NaN,gmail.com,NaN,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2987002,0,86469,59.0,W,4663,490.0,150.0,visa,166.0,debit,330.0,87.0,287.0,NaN,outlook.com,NaN,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2987003,0,86499,50.0,W,18132,567.0,150.0,mastercard,117.0,debit,476.0,87.0,NaN,NaN,yahoo.com,NaN,2.0,5.0,0.0,0.0,0.0,4.0,0.0,0.0,1.0,0.0,1.0,0.0,25.0,1.0,112.0,112.0,0.0,94.0,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2987004,0,86506,50.0,H,4497,514.0,150.0,mastercard,102.0,credit,420.0,87.0,NaN,NaN,gmail.com,NaN,1.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,70787.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,100.0,NotFound,NaN,-480.0,New,NotFound,166.0,NaN,542.0,144.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,New,NotFound,Android 7.0,samsung browser 6.2,32.0,2220x1080,match_status:2,T,F,T,T,mobile,SAMSUNG SM-G892A Build/NRD90M


In [6]:
trn['day']=trn['TransactionDT']//86400
for col in range(1,16):
  trn['d_'+str(col)]=trn['day']-trn['D'+str(col)]

In [7]:
print(trn.shape)

(590540, 450)


In [8]:
def second(data):
  return data[data.find('.'):]
def first(data):
  return data[:data.find('.')]
trn['P_emaildomain'].fillna('nan',inplace=True)
trn['R_emaildomain'].fillna('nan',inplace=True)
trn['P_emaildomain_first']=trn['P_emaildomain'].apply(first)
trn['P_emaildomain_second']=trn['P_emaildomain'].apply(second)
trn['R_emaildomain_first']=trn['R_emaildomain'].apply(first)
trn['R_emaildomain_second']=trn['R_emaildomain'].apply(second)

In [9]:
trn=trn.drop(['TransactionID','TransactionDT','isFraud'],1)
cat=list(trn.select_dtypes(include=object))
trn[cat]=trn[cat].fillna('nan')
trn=trn.fillna(-999)

In [10]:
trn=trn.loc[trn['D1']<trn['day']]

In [11]:
trn=trn.loc[trn['D1']<trn['day']].reset_index(drop=True)
import random
cols=['C13','C1','C11','C2','C9','id_31','id_02','id_05','id_33','id_20','id_12','D2','D3','D11','D4','D8','d_3','d_2','d_4','d_8','d_10','d_11','d_1','addr1','P_emaildomain','card1','card2','card5','dist2','ProductCD','TransactionAmt','V307']
random.shuffle(cols)

In [12]:
print(len(set(cols)))
print(len(cols))

32
32


In [13]:
from tqdm import tqdm
import random
def group(df,cols):
  df['id']=''
  for col in cols:
    df['id']+=df[col].astype(str)
  result=0
  for _,sdo in df.groupby(['id']):
    tmp=0
    tmp+=sdo['TransactionAmt'].values.sum()
    tmp-=sdo.tail(1)['V307'].values
    if tmp>0:
      result+=tmp
    else:
      tmp-=result
  return result
trn=trn[cols]
cols=list(trn)
cols.remove('TransactionAmt')
cols.remove('V307')
random.shuffle(cols)
ans=0
ls=[]
dk={}
for en,col in enumerate(tqdm(cols)):
  try:
    ls.append(col)
    n_ans=group(trn,ls)
    dk[col]=ans-n_ans
    if ans!=0:
      if n_ans>=ans:
        ls.remove(col)
      else:
        ans=n_ans
    else:
      ans=n_ans
  except:
    print(col)
    continue

100%|██████████| 30/30 [1:21:11<00:00, 162.40s/it]


In [14]:
ans

array([39494992.60837138])

In [15]:
df=pd.DataFrame(dk).T
df['rank']=list(range(30))
df.sort_values(by=0,ascending=False)

,0,rank
D4,4.955864e+06,2
d_2,4.479835e+06,4
D11,3.505296e+06,3
card1,3.286657e+06,5
d_11,1.015821e+06,9
addr1,5.090539e+05,1
d_3,4.359392e+05,10
id_12,3.501464e+05,6
d_8,2.251908e+05,11
id_05,2.175505e+05,8


In [16]:
len(ls)

22

In [17]:
ls

['C2',
 'addr1',
 'D4',
 'D11',
 'd_2',
 'card1',
 'id_12',
 'D2',
 'id_05',
 'd_11',
 'd_3',
 'd_8',
 'id_33',
 'd_1',
 'd_10',
 'card5',
 'D3',
 'C13',
 'C9',
 'id_31',
 'P_emaildomain',
 'C1']